In [ ]:
# default_exp ads

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#hide
!pip install -qU nbdev fastcore

     |████████████████████████████████| 61kB 3.1MB/s 


In [ ]:
#hide
!pip install -qU gspread facebook_business pycountry python-facebook-api

     |████████████████████████████████| 1.0MB 6.5MB/s 
     |████████████████████████████████| 10.1MB 36.4MB/s 
     |████████████████████████████████| 51kB 6.2MB/s 
     |████████████████████████████████| 81kB 8.8MB/s 


In [ ]:
#hide
from pathlib import Path
path = Path("/content/drive/My Drive/upwork/Martin followers/tomorrowswig_followers")
%cd "{path}"

/content/drive/My Drive/upwork/Martin followers/tomorrowswig_followers


In [ ]:
#hide
#not deps but we need them to use nbdev and run tests
from nbdev import * 
from nbdev.showdoc import *
from fastcore.test import *

## Facebook Auth

In [ ]:
#hide
%%capture

import yaml
with open(path.parent/"facebook_creds.yml") as f:
    data = yaml.safe_load(f)
    %env APP_ID {data.get("APP_ID")}
    %env APP_SECRET {data.get("APP_SECRET")}
    %env TOKEN {data.get("TOKEN")}
    %env AD_ACC_ID {data.get("AD_ACC_ID")}
    %env USER_ID {data.get("USER_ID")}

In [ ]:
#export
from tomorrowswig_followers.core import *
from tomorrowswig_followers.followers import get_new_followers

In [ ]:
#export
import os
from typing import *

from datetime import datetime, timedelta
from functools import partial

import pandas as pd
import numpy as np
from facebook_business.api import FacebookAdsApi
from facebook_business.adobjects.adaccount import AdAccount
import pycountry

AD_ACC_ID = os.environ.get("AD_ACC_ID")

In [ ]:
APP_ID = os.environ.get("APP_ID")
APP_SECRET = os.environ.get("APP_SECRET")
TOKEN = os.environ.get("TOKEN")
USER_ID = os.environ.get("USER_ID")

## Ads reporting

In [ ]:
#export
FacebookAdsApi.init(APP_ID, APP_SECRET, TOKEN)
account = AdAccount(AD_ACC_ID)

In [ ]:
datetime.fromtimestamp(1595946718)

datetime.datetime(2020, 7, 28, 14, 31, 58)

In [ ]:
datetime(2020,7,20,0,0).strftime('%s')

'1595203200'

In [ ]:
insights = account.get_insights(fields=[
    "ad_name",
    "clicks",
    "cpc",
    "ctr",
    "cpm",
    "cost_per_action_type",
    # AdsInsights.Field.cost_per_one_thousand_ad_impression, permissions
    "spend",
    # "adset_budget_value", permissions
    "impressions",
    "reach",
    "actions",
    "video_avg_time_watched_actions",
    "video_p50_watched_actions",
    "video_p75_watched_actions",
    "video_p95_watched_actions",

], params={
    'level': "ad",
    'date_preset': "yesterday",
    # "date_start": "1595116800",
    # "date_stop": "1595203200"
})


In [ ]:
def get_action(cell, name):
    action = [a for a in cell if a["action_type"] == name]
    if action:
        return action[0]["value"]
    return 0

In [ ]:
# fields = [f for f in AdsInsights.Field.__dict__ if not f.startswith("__")]

In [ ]:
# account.get_insights(fields=[fields[53]], params={
#     'level': AdsInsights.Level.ad,
#     'date_preset': AdsInsights.DatePreset.last_3d,
# })[-1]

In [ ]:
# account.get_insights(fields=fields[:53], params={
#     'level': AdsInsights.Level.ad,
#     'date_preset': AdsInsights.DatePreset.last_3d,
# })

In [ ]:
# account.get_insights(
#     # fields=fields[:53],
#     fields = ["actions"],
#     params={
#     'level': "ad",
#     'date_preset': "this_week_sun_today",
# })

In [ ]:
#export
def get_dif(df: pd.DataFrame) -> pd.DataFrame:
    change_df = df.diff(axis=1, periods=-1)
    change_df = change_df.fillna(0).astype(int)
    return change_df

In [ ]:
#export
# "Relevance Score",
COLUMNS = ["Ad Name", "New Followers", "% (Clicks)", "% (Impressions)", "Cost Per Follow",
           "Clicks (All)", "Link Clicks", "CPC (All)" , "CPC (Cost per Link Click)",
           "CTR (All)", "CPM (Cost per 1,000 Impressions)", "Amount Spent (USD)", "Impressions", "Reach", "Post Reactions", "Post Shares",
           "Video Average Play Time", "Video Plays at 50%", "Video Plays at 75%", "Video Plays at 95%"]

In [ ]:
#export
get_insights = partial(account.get_insights, fields=[
        "ad_name",
        "clicks",
        "cpc",
        "ctr",
        "cpm",
        "cost_per_action_type",
        "spend",
        "impressions",
        "reach",
        "actions",
        "video_avg_time_watched_actions",
        "video_p50_watched_actions",
        "video_p75_watched_actions",
        "video_p95_watched_actions",
    ], params={
        'level': "ad",
        'date_preset': "yesterday",
    })

In [ ]:
#export
def get_action(cell, name):
    action = [a for a in cell if a["action_type"] == name]
    if action:
        return action[0]["value"]
    return 0

In [ ]:
#export
def get_followers_change(date: datetime) -> pd.DataFrame:
    history_df = get_df("History")
    new_followers = get_dif(history_df)
    followers_date = (date + timedelta(days=1)).strftime("%b %d %Y")
    new_followers = new_followers.iloc[:,new_followers.columns.str.startswith(followers_date)].replace(0, np.nan)
    return new_followers.dropna(axis=1, how="all").iloc[:, :1].dropna().astype(int)

In [ ]:
assert get_followers_change(datetime.strptime("2020-07-25", "%Y-%m-%d")).columns[0] == "Jul 26 2020                 2020-07-26 19:10:34"

In [ ]:
#export
def match_country(x):
    try:
        return pycountry.countries.search_fuzzy(x)[0].alpha_2
    except LookupError as e:
        return "not found"

In [ ]:
assert match_country("brasil") == "not found"
assert match_country("brazil") == "BR"

In [ ]:
#export
def add_country_total(df: pd.DataFrame) -> Tuple[pd.DataFrame, str]:
    dupes = df[df.index.duplicated(False)]
    if not len(dupes):
        return

    to_mean = ["cpc", "cpm", "ctr", "CPC (Cost per Link Click)"]
    to_sum = set(df.columns).difference(set(to_mean + ["ad_name"]))
    total = dupes.iloc[0,:].copy()
    index = dupes.index[0]
    total["ad_name"] = f"{index} total"
    total[to_mean] = dupes[to_mean].apply(np.mean)
    total[to_sum] = dupes[to_sum].astype(int).apply(sum)
    return df.append(total), index

In [ ]:
#export
def add_followers(df: pd.DataFrame, date: datetime):
    new_followers = get_followers_change(date)
    if not new_followers.values.size:
        new_followers["no change"] = 0
    df["New Followers"] = new_followers
    df.fillna(0, inplace=True)
    df["% (Clicks)"] = (df["New Followers"] / df["clicks"]* 100).round().astype(int)
    df["% (Impressions)"] = (df["New Followers"] / df["impressions"] * 100)
    df["Cost Per Follow"] = (df["spend"] / df["New Followers"])
    df.replace([np.inf], 0, inplace=True)

In [ ]:
def create_first_part():
    df = pd.DataFrame(get_insights())
    floats = ["cpc", "cpm", "ctr", "spend"]
    df[floats] = df[floats].astype(float).round(2)
    ints = ["clicks", "impressions", "reach"]
    df[ints] = df[ints].astype(int)
    index = df["ad_name"].apply(lambda x: match_country(x.split("-")[1].strip()))
    index.name = "countries"
    df.set_index(index, inplace=True)
    df.sort_values("ad_name", inplace=True)

    date = datetime.strptime(df["date_start"].values[0], "%Y-%m-%d")
    worksheet_name = date.strftime("%b %d %Y")

    df["Post Reactions"] = df["actions"].apply(partial(get_action, name="post_reaction"))
    df["Post Shares"] = df["actions"].apply(partial(get_action, name="post"))
    df["Link Clicks"] = df["actions"].apply(partial(get_action, name="link_click"))
    df["CPC (Cost per Link Click)"] = df["cost_per_action_type"].apply(partial(get_action, name="link_click")).astype(float)
    get_video_action = partial(get_action, name="video_view")
    df["Video Average Play Time"] = df["video_avg_time_watched_actions"].apply(get_video_action)
    df["Video Plays at 50%"] = df["video_p50_watched_actions"].apply(get_video_action)
    df["Video Plays at 75%"] = df["video_p75_watched_actions"].apply(get_video_action)
    df["Video Plays at 95%"] = df["video_p95_watched_actions"].apply(get_video_action)

    df.drop(columns=["actions", "cost_per_action_type", "date_start", "date_stop",
                        "video_avg_time_watched_actions"] +
            [f"video_p{p}_watched_actions" for p in [50,75,95]], inplace=True)
    
    dupes = add_country_total(df)
    add_followers(df, date)
    return df

In [ ]:
df = create_first_part()

In [ ]:
to_mean = ["cpc", "cpm", "ctr", "CPC (Cost per Link Click)", "Video Average Play Time",
            "% (Clicks)", "% (Impressions)", "Cost Per Follow"]
to_sum = set(df.columns).difference(set(to_mean + ["ad_name"]))
total = pd.Series(name="total", index=df.columns, dtype=float)

In [ ]:
total[to_mean] = df[to_mean].apply(np.mean)

In [ ]:
total[to_sum] = df[to_sum].astype(int).apply(sum)

In [ ]:
df.append(total)

,ad_name,clicks,cpc,cpm,ctr,impressions,reach,spend,Post Reactions,Post Shares,Link Clicks,CPC (Cost per Link Click),Video Average Play Time,Video Plays at 50%,Video Plays at 75%,Video Plays at 95%,New Followers,% (Clicks),% (Impressions),Cost Per Follow
countries,,,,,,,,,,,,,,,,,,,,
AU,1 - Australia - General Ad 2-D,89.0,0.190,3.32000,1.72000,5162.0,5162.0,17.15,8,0,47,0.364894,3,1144,507,332,0.0,0.0,0.0,0.0
CA,1 - Canada - Gen #2 - D,40.0,0.110,2.44000,2.16000,1848.0,1847.0,4.51,1,0,21,0.214762,3,431,181,123,0.0,0.0,0.0,0.0
US,1 - USA - Gen 2d - STORIES+FEED,52.0,0.210,2.94000,1.40000,3710.0,3700.0,10.90,2,3,33,0.330303,3,786,351,226,0.0,0.0,0.0,0.0
DE,2 - Germany - General Ad 2-D,29.0,0.090,2.31000,2.51000,1157.0,1156.0,2.67,0,1,17,0.157059,3,283,110,83,0.0,0.0,0.0,0.0
MX,2 - Mexico - General Ad #2-D,73.0,0.070,0.59000,0.89000,8201.0,8192.0,4.80,4,0,39,0.123077,2,1396,369,249,0.0,0.0,0.0,0.0
AR,3 - Argentina - General 2-D,40.0,0.090,0.40000,0.43000,9252.0,9134.0,3.73,0,2,24,0.155417,1,1402,286,170,0.0,0.0,0.0,0.0
not found,3 - Brasil - General Ad 2-D,35.0,0.050,0.49000,0.91000,3863.0,3846.0,1.90,1,0,19,0.100000,2,646,180,114,0.0,0.0,0.0,0.0
ID,3 - Indonesia - General Ad 2-D,116.0,0.030,0.32000,0.95000,12188.0,12188.0,3.96,3,4,57,0.069474,2,1812,574,379,0.0,0.0,0.0,0.0
total,NaN,474.0,0.105,1.60125,1.37125,45381.0,45225.0,44.00,19,10,257,0.189373,4.16652e+06,7900,2558,1676,0.0,0.0,0.0,0.0


In [ ]:
#export
def add_total(df: pd.DataFrame) -> pd.DataFrame:
    to_mean = ["cpc", "cpm", "ctr", "CPC (Cost per Link Click)", "Video Average Play Time",
                "% (Clicks)", "% (Impressions)", "Cost Per Follow"]
    to_sum = set(df.columns).difference(set(to_mean + ["ad_name"]))
    total = pd.Series(name="total", index=df.columns)
    total[to_mean] = df[to_mean].apply(np.mean)
    total[to_sum] = df[to_sum].astype(int).apply(sum)
    return df.append(total)

In [ ]:
#export
def get_insights_df(insights: List) -> Tuple[pd.DataFrame, str]:
    df = pd.DataFrame(insights)
    floats = ["cpc", "cpm", "ctr", "spend"]
    df[floats] = df[floats].astype(float).round(2)
    ints = ["clicks", "impressions", "reach"]
    df[ints] = df[ints].astype(int)
    index = df["ad_name"].apply(lambda x: match_country(x.split("-")[1].strip()))
    index.name = "countries"
    df.set_index(index, inplace=True)
    df.sort_values("ad_name", inplace=True)

    date = datetime.strptime(df["date_start"].values[0], "%Y-%m-%d")
    worksheet_name = date.strftime("%b %d %Y")

    df["Post Reactions"] = df["actions"].apply(partial(get_action, name="post_reaction"))
    df["Post Shares"] = df["actions"].apply(partial(get_action, name="post"))
    df["Link Clicks"] = df["actions"].apply(partial(get_action, name="link_click"))
    df["CPC (Cost per Link Click)"] = df["cost_per_action_type"].apply(partial(get_action, name="link_click")).astype(float)
    get_video_action = partial(get_action, name="video_view")
    df["Video Average Play Time"] = df["video_avg_time_watched_actions"].apply(get_video_action)
    df["Video Plays at 50%"] = df["video_p50_watched_actions"].apply(get_video_action)
    df["Video Plays at 75%"] = df["video_p75_watched_actions"].apply(get_video_action)
    df["Video Plays at 95%"] = df["video_p95_watched_actions"].apply(get_video_action)

    df.drop(columns=["actions", "cost_per_action_type", "date_start", "date_stop",
                     "video_avg_time_watched_actions"] +
            [f"video_p{p}_watched_actions" for p in [50,75,95]], inplace=True)
    
    dupes = add_country_total(df)
    add_followers(df, date)
    df = add_total(df)
    to_round = ["Cost Per Follow", "% (Impressions)", "CPC (Cost per Link Click)"]
    df[to_round] = df[to_round].round(2)
    df.rename(columns={"ad_name": "Ad Name", "clicks": "Clicks (All)", "cpc": "CPC (All)",
                   "ctr": "CTR (All)", "cpm": "CPM (Cost per 1,000 Impressions)",
                   "reach": "Reach", "impressions": "Impressions", "spend": "Amount Spent (USD)"}, inplace=True)
    return df[COLUMNS], worksheet_name

In [ ]:
insights = get_insights()
df, worksheet_name = get_insights_df(insights)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
add_stats = pd.Series(dtype=float)
total_new_followers = df.loc["total", "New Followers"]
spent = df.loc["total", "Amount Spent (USD)"]
add_stats["Conversion Rate"] = total_new_followers / df.loc["total", "Clicks (All)"]
add_stats["Cost of New Follower"] = spent / total_new_followers

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in double_scalars
  """


In [ ]:
add_stats["Total Followers"] = get_new_followers()[1]
add_stats["Random Followers"] = add_stats["Total Followers"] - total_new_followers
add_stats["Cost of New Follower per Total"] = spent / add_stats["Total Followers"]

In [ ]:
add_stats["Unfollowers"] = add_stats["Total Followers"] - total_new_followers

In [ ]:
add_stats["Unconversion Rate"] = total_new_followers / df.loc["total", "Clicks (All)"]

In [ ]:
add_stats["Followers from ads guess"] = total_new_followers + add_stats["Random Followers"] - add_stats["Unfollowers"]
add_stats["Real cost guess"] = spent / add_stats["Followers from ads guess"]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in double_scalars
  


In [ ]:
#export
def more_stats(df: pd.DataFrame) -> pd.Series:
    add_stats = pd.Series(dtype=float)
    total_new_followers = df.loc["total", "New Followers"]
    spent = df.loc["total", "Amount Spent (USD)"]
    add_stats["Conversion Rate"] = total_new_followers / df.loc["total", "Clicks (All)"]
    add_stats["Cost of New Follower"] = spent / total_new_followers
    add_stats["Total Followers"] = get_new_followers()[1]
    add_stats["Random Followers"] = add_stats["Total Followers"] - total_new_followers
    add_stats["Cost of New Follower per Total"] = spent / add_stats["Total Followers"]
    add_stats["Unfollowers"] = add_stats["Total Followers"] - total_new_followers
    add_stats["Unconversion Rate"] = total_new_followers / df.loc["total", "Clicks (All)"]
    add_stats["Followers from ads guess"] = total_new_followers + add_stats["Random Followers"] - add_stats["Unfollowers"]
    add_stats["Real cost guess"] = spent / add_stats["Followers from ads guess"]
    return add_stats

In [ ]:
#export

def create_insights(event: Dict = None, context=None,) -> str:
    insights = get_insights()
    df, worksheet_name = get_insights_df(insights)
    add_stats = more_stats(df)
    empty = pd.Series(name="", dtype=str)
    df = df.append([empty, empty, empty])
    write_df(df, f"{worksheet_name}")
    write_series(add_stats, f"{worksheet_name}", loc=f"A{len(df)}")
    return f"Created insights in '{worksheet_name}'"

In [ ]:
create_insights()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in double_scalars
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in double_scalars
  


"Created insights in 'Jul 30 2020'"

In [ ]:
## TODO
## relevance score

## Next
# Match country - rename ads?

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 01_followers.ipynb.
Converted 02_ads.ipynb.
Converted index.ipynb.
